In [1]:
import boto3

In [2]:
# Document
s3BucketName = "aws-textract-multi-column-pdf"
documentName = "Scanned - Multi-column Single Page Document.pdf"

In [3]:
# Amazon Textract client
textract = boto3.client('textract')

In [4]:
# Call Amazon Textract
response = textract.detect_document_text(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    })

#print(response)

In [5]:
# Detect columns and print lines
columns = []
lines = []
for item in response["Blocks"]:
      if item["BlockType"] == "LINE":
        column_found=False
        for index, column in enumerate(columns):
            bbox_left = item["Geometry"]["BoundingBox"]["Left"]
            bbox_right = item["Geometry"]["BoundingBox"]["Left"] + item["Geometry"]["BoundingBox"]["Width"]
            bbox_centre = item["Geometry"]["BoundingBox"]["Left"] + item["Geometry"]["BoundingBox"]["Width"]/2
            column_centre = column['left'] + column['right']/2

            if (bbox_centre > column['left'] and bbox_centre < column['right']) or (column_centre > bbox_left and column_centre < bbox_right):
                #Bbox appears inside the column
                lines.append([index, item["Text"]])
                column_found=True
                break
        if not column_found:
            columns.append({'left':item["Geometry"]["BoundingBox"]["Left"], 'right':item["Geometry"]["BoundingBox"]["Left"] + item["Geometry"]["BoundingBox"]["Width"]})
            lines.append([len(columns)-1, item["Text"]])

In [6]:
print(len(columns))

2


I added the "reverse=True" as a parameter in line.sort()

In [7]:
lines.sort(key=lambda x: x[0], reverse=True)
for line in lines:
    print (line[1])

Neque porro quisquam est
qui dolorem ipsum quia
dolor sit amet,
consectetur, adipisci
velit
Lorem ipsum dolor sit amet, consectetur
adipiscing elit, sed do eiusmod tempor
incididunt ut labore et dolore magna aliqua. Ut
enim ad minim veniam, quis nostrud
exercitation ullamco laboris nisi ut aliquip ex
ea commodo consequat. Duis aute irure dolor
in reprehenderit in voluptate velit esse cillum
dolore eu fugiat nulla pariatur. Excepteur sint
occaecat cupidatat non proident, sunt in culpa
qui officia deserunt mollit anim id est laborum.
Sed ut perspiciatis unde omnis iste natus error
sit voluptatem accusantium doloremque
laudantium, totam rem aperiam, eaque ipsa
quae ab illo inventore veritatis et quasi
architecto beatae vitae dicta sunt explicabo.
Nemo enim ipsam voluptatem quia voluptas
sit aspernatur aut odit aut fugit, sed quia
consequuntur magni dolores eos qui ratione
voluptatem sequi nesciunt. Neque porro
quisquam est, qui dolorem ipsum quia dolor
sit amet, consectetur, adipisci veli